<a href="https://colab.research.google.com/github/ricardoskewes/68610finalprojectgroup48/blob/colab_branch/6861finalproject_group48.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.8610 Final Project FA24
Group 48

Katie Kostecki, Anthony Daoud, Ricardo Skewes, Aaron Zhu

**Loading in Dataset (stored in Github repo) and Data Preprocessing**

In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
rm -rf 68610finalprojectgroup48
git clone https://github.com/ricardoskewes/68610finalprojectgroup48.git

Cloning into '68610finalprojectgroup48'...


In [ ]:
import sys
sys.path.append("/content/68610finalprojectgroup48")

!pip install sacrebleu simpletransformers



In [ ]:


import csv
import itertools as it
import numpy as np
#import sklearn.decomposition
from tqdm import tqdm
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
#from torch.utils import data
#import math
#import sacrebleu
#from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
#from sklearn.linear_model import LogisticRegression
#import sklearn.linear_model
#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torch_data
import pandas as pd
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt

from transformers import set_seed, AutoModelForCausalLM, AutoTokenizer

set_seed(0) # to make outputs deterministic
model_name = 'openai-community/gpt2'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# prompt: import DataHandler.py
import sys
sys.path.append('/content/68610finalprojectgroup48/')


In [ ]:
### TweepFake (CSVs)
data = []
with open("/content/68610finalprojectgroup48/tweepfake/data/splits/train.csv") as reader:
  csvreader = csv.reader(reader, delimiter=';')
  next(csvreader)
  for screen_name, text, account_type, class_type in csvreader: # see https://www.kaggle.com/datasets/mtesconi/twitter-deep-fake-text for details
    data.append((text, screen_name, account_type))
print(f"Number of training tweets: {len(data)}")


Number of training tweets: 20712


In [ ]:


with open("/content/68610finalprojectgroup48/reuter+50+50/C50train/ToddNissen/107276newsML.txt", 'r') as file:
  content = file.read()
print(content)

The engines have started in the race to succeed Alex Trotman for the top office at Ford Motor Co.
The company on Thursday promoted two longtime Ford executives to positions that will allow them to prove if they are worthy of occupying one or more of the top spots at Detroit's No. 2 automaker.  
Edward Hagenlocker, 56, was promoted to vice chairman from executive vice president, and put in charge of a new auto components organization with 75,000 employees and revenues of $14 billion.
Jacques Nasser, 48, previously group vice president in charge of product development, was named to Hagenlocker's old job of heading up Ford Automotive Operations (FAO), the company's core car and truck business.  
Speculation has swirled for months about how Ford's board of directors plans to handle the succession of Trotman, who currently holds the titles of chairman, chief executive officer and president. Trotman reaches the mandatory retirement age of 65 in July 1998, but the board can ask him to stay on

# Tweepfake

## Baseline model aka BERT*(tweet)
This is the model that comes from the paper.


In [ ]:
#%tensorflow_version 2.x

# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)
# import sys
# projectDir = "/content/drive/My Drive/code/git/tweepfake_deepfake_text_detection"
# sys.path.insert(0, projectDir)

# random_state = 523 # Fixed seed for replicability of randomic operations.
# resultsDir = projectDir+"/data/results"
# %%writefile setup.sh

# export CUDA_HOME=/usr/local/cuda-10.1
# git clone https://github.com/NVIDIA/apex
# pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
# !sh setup.sh

In [ ]:
import sys
projectDir = "/content/68610finalprojectgroup48/tweepfake"
sys.path.insert(0, projectDir)
resultsDir = projectDir+"/data/results"
random_state = 523 # Fixed seed for replicability of randomic operations.

In [ ]:
from DataHandler import DataHandler

csvTrainDataset = projectDir+"/data/splits/train.csv"
csvValDataset = projectDir+"/data/splits/validation.csv"
csvTestDataset = projectDir+"/data/splits/test.csv"

bertDir = projectDir+"/data/encoded/bert"
dh = DataHandler()
dfTrain = dh.readCSVData(csvTrainDataset)
dfVal = dh.readCSVData(csvValDataset)
dfTest = dh.readCSVData(csvTestDataset)

In [ ]:
dfTrain

,screen_name,text,account.type,class_type
0,imranyebot,YEA now that note GOOD,bot,others
1,zawvrk,Listen to This Charming Man by The Smiths htt...,human,human
2,zawarbot,wish i can i would be seeing other hoes on the...,bot,others
3,ahadsheriffbot,The decade in the significantly easier schedul...,bot,others
4,kevinhookebot,"""Theim class=\""alignnone size-full wp-image-60...",bot,rnn
...,...,...,...,...
20707,AINarendraModi,Met on the Abversion of our science for the co...,bot,rnn
20708,AINarendraModi,Land for their during the opportunity to the p...,bot,rnn
20709,DeepDrumpf,@TayandYou doesn't have a clue. You're right. ...,bot,rnn
20710,jaden,Me And My Bestie https://t.co/vPq2iDkWZm,human,human


In [ ]:
# Select interesting columns for this study.
dfTrainDataset = dfTrain[["screen_name", "text", "account.type"]]
dfValDataset = dfVal[["screen_name", "text", "account.type"]]
dfTestDataset = dfTest[["screen_name", "text", "account.type"]]

In [ ]:
dfTrainDataset

,screen_name,text,account.type
0,imranyebot,YEA now that note GOOD,bot
1,zawvrk,Listen to This Charming Man by The Smiths htt...,human
2,zawarbot,wish i can i would be seeing other hoes on the...,bot
3,ahadsheriffbot,The decade in the significantly easier schedul...,bot
4,kevinhookebot,"""Theim class=\""alignnone size-full wp-image-60...",bot
...,...,...,...
20707,AINarendraModi,Met on the Abversion of our science for the co...,bot
20708,AINarendraModi,Land for their during the opportunity to the p...,bot
20709,DeepDrumpf,@TayandYou doesn't have a clue. You're right. ...,bot
20710,jaden,Me And My Bestie https://t.co/vPq2iDkWZm,human


In [ ]:
X_train_all = dfTrainDataset.drop(columns=['screen_name'])
X_train_all.columns = ["text", "label"]

X_val_all = dfValDataset.drop(columns=['screen_name'])
X_val_all.columns = ["text", "label"]

X_test_all = dfTestDataset.drop(columns=['screen_name'])
X_test_all.columns = ["text", "label"]

dictLabels = {"human":0, "bot":1}
dictLabelsReverse = {0:"human", 1: "bot"}

X_train_all["label"] = X_train_all["label"].apply(lambda x: dictLabels[x])
X_val_all["label"] = X_val_all["label"].apply(lambda x: dictLabels[x])
X_test_all["label"] = X_test_all["label"].apply(lambda x: dictLabels[x])
y_train = X_train_all["label"]
y_val = X_val_all["label"]
y_test = X_test_all["label"]

In [ ]:
train_labels = y_train.tolist()
val_labels = y_val.tolist()
test_labels = y_test.tolist()

In [ ]:
from simpletransformers.classification import ClassificationModel

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-cased', args={'fp16': False, 'num_train_epochs': 3, "overwrite_output_dir":True}) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(X_train_all)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/2589 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/2589 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/2589 [00:00<?, ?it/s]

(7767, 0.24892444256980903)

In [ ]:
# Evaluate the model
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(X_test_all, acc=sklearn.metrics.accuracy_score, f1=sklearn.metrics.f1_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
result

{'mcc': 0.7932092395575824,
 'accuracy': 0.8964034401876466,
 'f1_score': 0.8963741579741085,
 'tp': 1168,
 'tn': 1125,
 'fp': 153,
 'fn': 112,
 'auroc': 0.9638295921361503,
 'auprc': 0.9635793725386573,
 'acc': 0.8964034401876466,
 'f1': 0.8981161091887736,
 'eval_loss': 0.5036312404733437}

In [ ]:
predictions = model.predict(X_test_all["text"].to_list())

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
dfTest['predictions'] = [dictLabelsReverse[x] for x in predictions[0]]
dfTestt =dfTest.drop(columns=['class_type'])


In [ ]:
res1 = dfTestt[dfTestt['account.type'] != dfTestt['predictions']]

In [ ]:
dfTestt[dfTestt['account.type'] != dfTestt['predictions']][:50]


,screen_name,text,account.type,predictions
8,dril,"ive been known to look at 2, even 3 emails , a...",human,bot
35,imranye,may allah please grant my bank account shifaa,human,bot
36,dril,"i have created more jobs, than every Weed Smok...",human,bot
38,ninjasexparty,porn movie idea: An Inspector Balls,human,bot
52,zawarbot,damn who wants to give you harambe ...... <U+0...,bot,human
69,dril,the Clitoral pussy,human,bot
75,kevinhooke,Your ignorance is not a valid reason that a be...,human,bot
79,imranyebot,my boys hype me and,bot,human
84,dril,consider this: the time and energy it took for...,human,bot
110,zawarbot,don’t gotta be a nut to sleeping early?,bot,human


**Obtaining Simple Word Embeddings** (e.g., bag-of-words)

**Obtaining Transformer Embeddings** ("stylistic?")

**Concatenating Embeddings and Passing Through Feed-Forward Neural Net**

**Siamese Network and Classification**